# Бобкова Ксения дз2

Был сформирован небольшой текст из разных источников.
Сложными случаями для автоматической разметки показались части речи, которые не везде выделяются, например, причастие, деепричастие. Кроме того, подклассы местоимений, которые не всегда одназначно можно выделить, союзы, которые при разметке могут быть разделены на несколько частей.

**На следующий день, в обед, Зеленин сидел в чайной и смотрел в окно на бескрайнюю ширь озера. Было ветрено, мрачно, ходуном ходил тёмно-серый взлохмаченный горизонт.
Чайной, темно-серый
Некому было заняться?
Некому
Сейчас Зеленин чувствовал себя словно древний мореплаватель, только что миновавший Геркулесовы столбы.
Вот представьте, опять-таки, общественный транспорт — автобус только что остановился, открылась передняя дверь с турникетом, чтобы впустить садящихся, а потом уже (чтобы не было зайцев) откроются остальные двери, чтобы выпустить сходящих.
А вот этого — ой, как не хватает современному обществу.
Если научить учащегося, в первую очередь, вот такой вот простой психологии — переводить гневные выкрики в спокойные замечания, то учащийся скорее поймёт всю вредность профессии преподавателя и просто перестанет его раздражать, или научится, хотя бы, не реагировать на обидные слова.
Мужчины плохо понимают женщин, но и сами женщины часто не понимают самих себя.
Женщины, обладая терпеливостью и усидчивостью, способны выполнять тонкую и монотонную работу с заданной программой действий 
«Мой брат в Москве, мы Россию любим, на русских бизнеса нет», — убеждал меня такой «впариватель» возле мечети Байязет в Стамбуле
Мы духовны, когда беседуем о чём-то дорогом с товарищем, доверяя его уму и сердцу.
А увенчанный славой военачальник был желанным гостем в доме своей племянницы. 
Несмотря на непогоду, делает сквозь льды проходы.
Количество людей, уровень которых позволяет делать последовательной перевод с языка и на язык еще меньше, считается, что таких готовят в языковых ВУЗах.**


В первую очередь, было необходимо записать исходный текст в csv таблицу. Отмечу, что текст в файле уже был разбит по предложениям (строчка - 1 предложение), поэтому требовалось только токенизировать его, а также присвоить id каждому слову для конкретного предложения. Другими словами, все слова конкретного предложения должны иметь один id.

In [ ]:
from nltk.tokenize import word_tokenize
import pandas as pd
from itertools import chain, repeat

all_sentences = []
with open("sentences.txt","rt", encoding="UTF-8") as f1:
    for line in f1:
        all_sentences.append(line.rstrip())

all_tokens_sent = []
for sentence in all_sentences:
    tokens = [w.lower() for w in word_tokenize(sentence) if w.isalpha()]
    all_tokens_sent.append(tokens)

#all tokens
all_tokens = list(chain.from_iterable(all_tokens_sent))


#get id of sentences
id_sentences = []
for i in range(len(all_tokens_sent)):
    dl = len(all_tokens_sent[i])
    id_sentences.extend(repeat(i, dl))

#create a dict
dict_to_table = {}
dict_to_table["Word"] = all_tokens
dict_to_table["id_sent"] = id_sentences
#
#create a table
df = pd.DataFrame(dict_to_table)
df.to_csv('table.csv', mode='a', index= False , header= True )

Затем вручную таблица была размечена. После же были использованы 3 POS теггера для русского языка, результаты записаны в таблицу по колонкам.

In [ ]:
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import pandas as pd
import stanza
#stanza.download('ru')



df = pd.read_csv('table.csv')

#pymorhy - Pos-tag1
morph = MorphAnalyzer()
tags_pymorhy = []
for token in df["Word"]:
     t = morph.parse(token)
     tag = t[0].tag.POS
     tags_pymorhy.append(tag)
        
 #add to csv file
df['Pos-tag1'] = tags_pymorhy
df.to_csv('table.csv', index=False)

#mystem - Pos-tag2
mystem = Mystem()
tags_mystem = []
for token in df["Word"]:
     m = mystem.analyze(token)
     gr = m[0]["analysis"][0]["gr"]
     pos = gr.split("=")[0].split(",")[0]
     tags_mystem.append(pos)

# #add to csv file
df['Pos-tag2'] = tags_mystem
df.to_csv('table.csv', index=False)


#stanza - Pos-tag3
ppln = stanza.Pipeline('ru', processors='tokenize,pos')
tags_stanza = []
for token in df["Word"]:
     doc = ppln(token)
     d = doc.to_dict()
     s = d[0][0]["upos"]
     tags_stanza.append(s)

#add to csv file
 df['Pos-tag3'] = tags_stanza
 df.to_csv('table.csv', index=False)